##### Copyright 2021 The Cirq Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://quantumai.google/cirq/qcvv/xeb_coherent_noise>"><img src="https://quantumai.google/site-assets/images/buttons/quantumai_logo_1x.png" />View on QuantumAI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/quantumlib/Cirq/blob/master/docs/qcvv/xeb_coherent_noise.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/quantumlib/Cirq/blob/master/docs/qcvv/xeb_coherent_noise.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/Cirq/docs/qcvv/xeb_coherent_noise.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/download_icon_1x.png" />Download notebook</a>
  </td>
</table>

In [ ]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")

# XEB and Coherent Error

In [ ]:
import numpy as np

import cirq
from cirq.contrib.svg import SVGCircuit

## Set up Random Circuits

We create a set of 10 random, two-qubit `circuits` which uses `SINGLE_QUBIT_GATES` to randomize the circuit and `SQRT_ISWAP` as the entangling gate. We will ultimately truncate each of these circuits according to `cycle_depths`. Please see [the XEB Theory notebook](./xeb_theory.ipynb) for more details.

In [ ]:
exponents = np.linspace(0, 7/4, 8)
exponents

In [ ]:
import itertools
SINGLE_QUBIT_GATES = [
    cirq.PhasedXZGate(x_exponent=0.5, z_exponent=z, axis_phase_exponent=a)
    for a, z in itertools.product(exponents, repeat=2)
]
SINGLE_QUBIT_GATES[:10], '...'

In [ ]:
import cirq_google as cg
from cirq.experiments import random_quantum_circuit_generation as rqcg

q0, q1 = cirq.LineQubit.range(2)

In [ ]:
# Make long circuits (which we will truncate)
n_circuits = 10
circuits = [
    rqcg.random_rotations_between_two_qubit_circuit(
        q0, q1, 
        depth=100, 
        two_qubit_op_factory=lambda a, b, _: cirq.SQRT_ISWAP(a, b), 
        single_qubit_gates=SINGLE_QUBIT_GATES)
    for _ in range(n_circuits)
]

In [ ]:
# We will truncate to these lengths
max_depth = 100
cycle_depths = np.arange(3, max_depth, 9)
cycle_depths

## Emulate coherent error

We request a $\sqrt{i\mathrm{SWAP}}$ gate, but the quantum hardware may execute something subtly different. Therefore, we move to a more general 5-parameter two qubit gate, `cirq.PhasedFSimGate`. 

This is the general excitation-preserving two-qubit gate, and the unitary matrix of PhasedFSimGate(θ, ζ, χ, γ, φ) is:

    [[1,                       0,                       0,            0],
     [0,    exp(-iγ - iζ) cos(θ), -i exp(-iγ + iχ) sin(θ),            0],
     [0, -i exp(-iγ - iχ) sin(θ),    exp(-iγ + iζ) cos(θ),            0],
     [0,                       0,                       0, exp(-2iγ-iφ)]].

This parametrization follows eq (18) in https://arxiv.org/abs/2010.07965. Please read the docstring for `cirq.PhasedFSimGate` for more information.

With the following code, we show how `SQRT_ISWAP` can be written as a specific `cirq.PhasedFSimGate`.

In [ ]:
sqrt_iswap_as_phased_fsim = cirq.PhasedFSimGate.from_fsim_rz(
    theta=-np.pi/4, phi=0, 
    rz_angles_before=(0,0), rz_angles_after=(0,0))
np.testing.assert_allclose(
    cirq.unitary(sqrt_iswap_as_phased_fsim),
    cirq.unitary(cirq.SQRT_ISWAP),
    atol=1e-8
)

We'll also create a perturbed version. Note the $\pi/16$ `phi` angle:

In [ ]:
perturbed_sqrt_iswap = cirq.PhasedFSimGate.from_fsim_rz(theta=-np.pi/4, phi=np.pi/16,
                                                        rz_angles_before=(0,0), rz_angles_after=(0,0))
np.round(cirq.unitary(perturbed_sqrt_iswap), 3)

We'll use this perturbed gate along with the `GateSubstitutionNoiseModel` to create simulator which has a constant coherent error. Namely, each `SQRT_ISWAP` will be substituted for our perturbed version.

In [ ]:
def _sub_iswap(op):
    if op.gate == cirq.SQRT_ISWAP:
        return perturbed_sqrt_iswap.on(*op.qubits)
    return op

noise = cirq.devices.noise_model.GateSubstitutionNoiseModel(_sub_iswap)
noisy_sim = cirq.DensityMatrixSimulator(noise=noise)

## Run the benchmark circuits

We use the function `sample_2q_xeb_circuits` to execute all of our circuits at the requested `cycle_depths`.

In [ ]:
from cirq.experiments.xeb_sampling import sample_2q_xeb_circuits
sampled_df = sample_2q_xeb_circuits(sampler=noisy_sim, circuits=circuits, 
                                    cycle_depths=cycle_depths, repetitions=10_000)
sampled_df.head()

## Compute fidelity assuming `SQRT_ISWAP`

In contrast to the XEB Theory notebook, here we only have added coherent error (not depolarizing). Nevertheless, the random, scrambling nature of the circuits shows circuit fidelity decaying with depth (at least when we assume that we were trying to use a pure `SQRT_ISWAP` gate)

In [ ]:
from cirq.experiments.xeb_fitting import benchmark_2q_xeb_fidelities
fids = benchmark_2q_xeb_fidelities(sampled_df, circuits, cycle_depths)
fids.head()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

xx = np.linspace(0, fids['cycle_depth'].max())
plt.plot(xx, (1-5e-3)**(4*xx), label=r'Exponential Reference')

plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label='Perturbed fSim')

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth $d$')
plt.legend(loc='best')

## Optimize `PhasedFSimGate` parameters

We know what circuits we requested, and in this simulated example, we know what coherent error has happened. But in a real experiment, there is likely unknown coherent error that you would like to characterize. Therefore, we make the five angles in `PhasedFSimGate` free parameters and use a classical optimizer to find which set of parameters best describes the data we collected from the noisy simulator (or device, if this was a real experiment).

fids_opt = simulate_2q_xeb_fids(sampled_df, pcircuits, cycle_depths, param_resolver={'theta': -np.pi/4, 'phi': 0.1})

In [ ]:
import multiprocessing
pool = multiprocessing.get_context('spawn').Pool()

In [ ]:
from cirq.experiments.xeb_fitting import \
    parameterize_circuit, characterize_phased_fsim_parameters_with_xeb, SqrtISwapXEBOptions

options = SqrtISwapXEBOptions(
    characterize_theta=True, 
    characterize_phi=True,
    characterize_chi=False,
    characterize_gamma=False,
    characterize_zeta=False
)
p_circuits = [parameterize_circuit(circuit, options) for circuit in circuits]
res = characterize_phased_fsim_parameters_with_xeb(
    sampled_df, 
    p_circuits, 
    cycle_depths, 
    options, 
    pool=pool,
    xatol=1e-3,
    fatol=1e-3)

In [ ]:
xx = np.linspace(0, fids['cycle_depth'].max())
p_depol = 5e-3 # from above
plt.plot(xx, (1-p_depol)**(4*xx), label=r'Exponential Reference')
plt.axhline(1, color='grey', ls='--')

plt.plot(fids['cycle_depth'], fids['fidelity'], 'o-', label='Perturbed fSim')
plt.plot(res.fidelities_df['cycle_depth'], res.fidelities_df['fidelity'], 'o-', label='Refit fSim')

plt.ylabel('Circuit fidelity')
plt.xlabel('Cycle Depth')
plt.legend(loc='best')
plt.tight_layout()